# Setup Dependencies

In [1]:
import sys
!{sys.executable} -m pip install requests

In [2]:
import requests

# Evaluation

## Get information on available dmp loaders

In [3]:
dmp_loader_info = requests.get('http://localhost:8080/info/dmp-providers')
dmp_loader_info.json()

['JSON-FILE']

## Get information on available context providers

In [4]:
context_loader_info = requests.get('http://localhost:8080/info/context-providers')
context_loader_info.json()

['OPEN_AIRE', 'RE3DATA']

### Get information on available evaluators

In [5]:
eval_info = requests.get('http://localhost:8080/api/evaluation/info/evaluators')
eval_info.json()

[{'category': 'COMPLETENESS',
  'dimensions': ['DCS_COMPLETENESS', 'SCIENCE_EUROPE_EXTENSION_COMPLETENESS']},
 {'category': 'COMPLIANCE',
  'dimensions': ['DCS_COMPLIANCE', 'SCIENCE_EUROPE_GUIDELINE_COMPLIANCE']},
 {'category': 'FEASIBILITY', 'dimensions': ['ACCURACY', 'AVAILABILITY']},
 {'category': 'QUALITY_OF_ACTIONS', 'dimensions': ['FAIR']}]

### Call Evaluation Endpoint to produce all available measurements for the minimal DMP

In [6]:
json_data = {
    'dmpLoaderParameters': {
        'dmpLoader': 'JSON-FILE',
        'dmpIdentifier': 'dcs-repo-examples/ex8-dmp-minimal-content.json',
    },
    'dataLifecycle': 'PUBLISHED',
    'dimensions': [],
    'categories': []
}

headers = {
    'Content-Type': 'application/json'
}

response = requests.post('http://localhost:8080/api/evaluation/evaluate', headers=headers, json=json_data)

result = response.json()

In [7]:
result

{'dmpStoreId': 'f8c13d55-d2c3-4d2e-b8b6-5824ba8e6fbd',
 'evaluationId': '2a041369-f338-4638-bc41-3b30ee14ec9c',
 'measurements': [{'lifeCycleStage': {'title': 'PUBLISHED'},
   'isMeasurementOf': {'identifier': 'required_entity_or_property_existent',
    'description': 'Existence of a required entity or property according to the specification',
    'title': 'DCS Completeness',
    'inDimension': {'inCategory': {'title': 'COMPLETENESS'},
     'title': 'DCS_COMPLETENESS'},
    'applicableDMPLifeCycles': [{'title': 'PLANNING'}],
    'expectedDataType': 'http://www.w3.org/2001/XMLSchema#boolean',
    'metricTests': []},
   'computedOn': {'entity': 'dmp'},
   'value': True,
   'softwareAgent': {'title': 'Apache Jena SHACL Validator'},
   'testResults': []},
  {'lifeCycleStage': {'title': 'PUBLISHED'},
   'isMeasurementOf': {'identifier': 'dcs_multiplicity_metric',
    'description': 'Mutltiplicity of value in the DMP in accordance with the DCS application profile',
    'title': 'DCS Multipli

In [8]:
measurements = result["measurements"]
#evalDimensions = set(map(lambda x: x["isMeasurementOf"]["inDimension"]["title"], measurements))
evalDimensions = set([x["isMeasurementOf"]["inDimension"]["title"] for x in measurements])
evalDimensions

{'AVAILABILITY', 'DCS_COMPLETENESS', 'DCS_COMPLIANCE'}